In [0]:
!ls hyperliquid/utils/kafka_callbacks.py
!ls hyperliquid/utils/rate_limiter.py
!ls hyperliquid/utils/example_utils.py


In [0]:
!pip install aiokafka
!pip install kafka-python
dbutils.library.restartPython()


In [0]:
import asyncio
from asyncio import run_coroutine_threadsafe
import os
import json
from datetime import datetime
from typing import Any

from hyperliquid.utils import constants
from hyperliquid.utils import rate_limiter
from hyperliquid.utils import example_utils

from kafka import KafkaAdminClient
from kafka.admin import NewTopic
from kafka.errors import TopicAlreadyExistsError

# Import our Kafka callbacks
from hyperliquid.utils.kafka_callbacks import (
    ClickHouseFillKafka,
    ClickHouseOrderKafka,
    ClickHouseLiquidationsKafka,
    ClickHouseLiquidationsDevKafka
)

import yaml
import time
# USE_DEV = True
# start_time = int(time.time() * 1000)

def load_addresses_from_config(path="config_whales.yml") -> list[str]:
    try:
        with open(path, "r") as f:
            data = yaml.safe_load(f)
            print(f"Loaded addresses from {path}: {data}")
            return data.get("addresses", [])
    except Exception as e:
        print(f"[ERROR] Failed to load addresses from {path}: {e}")
        return []


# Set up your Kafka config
KAFKA_SERVER = "cthki8qfdq8asdnsm9gg.any.us-east-1.mpx.prd.cloud.redpanda.com:9092"
USERNAME = "alexei.jobfinder@gmail.com"
PASSWORD = "y0obC7dFiU3CJxcsCH4RwXtwEhaauf"
username = "alexei.jobfinder@gmail.com"
password = 'y0obC7dFiU3CJxcsCH4RwXtwEhaauf'
bootstrap_servers = 'cthki8qfdq8asdnsm9gg.any.us-east-1.mpx.prd.cloud.redpanda.com:9092'

fill_kafka_cb = None
order_kafka_cb = None
liquidation_kafka_cb = None

# # Instantiate the fill/order Kafka callbacks
# fill_kafka_cb = ClickHouseFillKafka(
#     bootstrap=KAFKA_SERVER,
#     username=USERNAME,
#     password=PASSWORD
# )

# order_kafka_cb = ClickHouseOrderKafka(
#     bootstrap=KAFKA_SERVER,
#     username=USERNAME,
#     password=PASSWORD
# )
# if USE_DEV:
#     liquidation_kafka_cb = ClickHouseLiquidationsDevKafka(
#         bootstrap=KAFKA_SERVER,
#         username=USERNAME,
#         password=PASSWORD
#     )
# else:
#     liquidation_kafka_cb = ClickHouseLiquidationsKafka(
#         bootstrap=KAFKA_SERVER,
#         username=USERNAME,
#         password=PASSWORD
#     )


rate_limiter = rate_limiter.HyperliquidRateLimiter()
event_loop = asyncio.get_event_loop()


def ensure_topics(bootstrap_servers, username, password, sasl_mechanism="SCRAM-SHA-256"):
    admin = KafkaAdminClient(
        bootstrap_servers=bootstrap_servers,
        security_protocol="SASL_SSL",
        sasl_mechanism=sasl_mechanism,
        sasl_plain_username=username,
        sasl_plain_password=password,
    )

    topics_to_create = [
        'orders',
        'fills',
        'liquidations_dev',
    ]

    existing_topics = admin.list_topics()

    new_topics = []
    for t in topics_to_create:
        if t not in existing_topics:
            new_topics.append(NewTopic(name=t, num_partitions=1, replication_factor=3))

    if new_topics:
        try:
            admin.create_topics(new_topics=new_topics)
            print("Created topics:", [nt.name for nt in new_topics])
        except TopicAlreadyExistsError:
            print("Some topics already exist.")
    else:
        print("All topics already exist")

    admin.close()



# Notation reference for Hyperliquid v0 API (nonstandard; subject to change in v1):
# Px   = Price
# Sz   = Size (in units of coin, i.e., base currency)
# Szi  = Signed size (positive for long, negative for short)
# Ntl  = Notional (USD amount, Px * Sz)
# Side = Side of trade/book: B = Bid = Buy, A = Ask = Short (aggressing side for trades)
# Asset = Integer representing the asset being traded
# Tif  = Time in force: GTC = Good 'til canceled, ALO = Add liquidity only, IOC = Immediate or cancel

#     {\"coin\":\"BTC\",\"px\":\"79823.0\",\"sz\":\"0.01126\",\"side\":\"A\",\"time\":1743963674808,\"startPosition\":\"0.01126\",\"dir\":\"Close Long\",\"closedPnl\":\"-6.768386\",\"hash\":\"0x766b22d7a6c15f8f4fef0421080ab00203730023705d3dd217a1dbfe8f96a41e\",\"oid\":84299475887,\"crossed\":true,\"fee\":\"0.224701\",\"tid\":602065776560497,\"liquidation\":{\"liquidatedUser\":\"0xa44481a6454f4fd0899e261aa941323f2b11a09b\",\"markPx\":\"79840.0\",\"method\":\"market\"},\"feeToken\":\"USDC\"},

# Notation reference for Hyperliquid v0 API (nonstandard; subject to change in v1):
# Px   = Price
# Sz   = Size (in units of coin, i.e., base currency)
# Szi  = Signed size (positive for long, negative for short)
# Ntl  = Notional (USD amount, Px * Sz)
# Side = Side of trade/book: B = Bid = Buy, A = Ask = Short (aggressing side for trades)
# Asset = Integer representing the asset being traded
# Tif  = Time in force: GTC = Good 'til canceled, ALO = Add liquidity only, IOC = Immediate or cancel
"""
Processes user fill messages, enriches with timing metadata, and writes to Kafka.

Timing fields added:
- `timestamp`: The original event time from the exchange, in milliseconds.
- `receipt_timestamp`: Wall-clock time (in seconds, float) when this event was handled by your system.
- `loop_timestamp`: Monotonic time (in seconds, float) when the event loop resumed this coroutine.
- `loop_delay_nanoseconds`: Delay (in nanoseconds) between wall-clock `receipt_timestamp` and event loop resume.
    → Measures how much time passed between the coroutine being scheduled vs. when it was actually resumed.
- `event_loop_delay_nanoseconds`: Delay (in nanoseconds) between exchange event time and the loop resume time.
    → Measures network delay, buffering, and event propagation latency.

These fields help debug event latency, monitor scheduling jitter, and quantify end-to-end processing delays.
"""

# async def handle_user_fills(msg, address):
#     await rate_limiter.acquire_address_action(address)
#     fills = msg.get("data", {}).get("fills", [])
#     for f in fills:
#         try:
#             ts = f.get("time")  # milliseconds since Unix epoch (exchange)
#             now = time.time()  # seconds (float), wall-clock
#             loop_timestamp_sec = asyncio.get_event_loop().time()  # seconds (float), monotonic loop time
#             loop_timestamp_ns = int(loop_timestamp_sec * 1_000_000_000)  # convert monotonic loop timestamp to nanoseconds

#             # Convert all timestamps to nanoseconds for delay calculations
#             receipt_timestamp_ns = int(now * 1_000_000_000)  # wall-clock to ns
#             ts_ns = int(ts * 1_000_000)  # exchange timestamp from ms to ns

#             # Calculate delays
#             loop_delay_nanoseconds = receipt_timestamp_ns - ts_ns  # time from exchange event to arrival at system (network latency approximation)
#             event_loop_delay_nanoseconds = loop_timestamp_ns - ts_ns  # exchange timestamp to processing (full latency)

#             # Extract liquidation info
#             liquidation = f.get("liquidation", {}) or {}
#             is_liquidation = 1 if liquidation else 0
#             liquidated_user = liquidation.get("liquidatedUser") if is_liquidation else -1
#             liquidation_method = liquidation.get("method") if is_liquidation else None
#             liquidation_mark_price = liquidation.get("markPx") if is_liquidation else -1

#             normalized_side = {"B": "buy", "A": "sell"}.get(f.get("side"), f.get("side"))

#             fill_data = {
#                 "exchange": "hyperliquid",
#                 "symbol": f.get("coin"),
#                 "side": normalized_side,
#                 "price": f.get("px"),
#                 "amount": f.get("sz"),
#                 "fee": f.get("fee"),
#                 "id": f.get("hash"),
#                 "order_id": f.get("oid"),
#                 "liquidity": f.get("crossed"),
#                 "type": f.get("dir"),
#                 "account": address,
#                 "timestamp": ts,  # exchange timestamp, ms
#                 "receipt_timestamp": now,  # wall-clock seconds
#                 "loop_timestamp": loop_timestamp_ns,  # monotonic loop timestamp, ns
#                 "loop_delay_nanoseconds": loop_delay_nanoseconds,
#                 "event_loop_delay_nanoseconds": event_loop_delay_nanoseconds,
#                 "is_liquidation": is_liquidation,
#                 "liquidated_user": liquidated_user,
#                 "liquidation_method": liquidation_method,
#                 "liquidation_mark_price": liquidation_mark_price,
#                 "raw": f,
#                 "raw_data": f,
#             }

#             print(f"[DEBUG] Fills data for address {address}: {fill_data}")
#             await fill_kafka_cb.write(fill_data)

"""
Processes user fill messages, enriches them with timing metadata, and writes them to Kafka.

Timing fields added:

timestamp: Exchange-generated event timestamp (Unix epoch, nanoseconds). Represents the original event time provided by the exchange.

receipt_timestamp: Wall-clock timestamp when the event reached your system (Unix epoch, nanoseconds). Indicates when your local machine received and began processing the event.

loop_timestamp: Monotonic timestamp at the moment the event loop resumed processing this coroutine (nanoseconds since event loop start). Used for measuring internal scheduling latency independent of the system clock.

loop_delay_nanoseconds: Latency between coroutine scheduling and event loop processing start, measured using monotonic timestamps. Reflects internal event loop scheduling delay (e.g., coroutine queuing delays). Typically expected to be very short (microseconds to low milliseconds).

event_loop_delay_nanoseconds: Network/event propagation latency from exchange timestamp to local system receipt timestamp. Reflects real-world latency including network delay and event buffering.

Real-world verified example:

{
  "timestamp": 1744089252941000000,          // exchange event time (~year 2025)
  "receipt_timestamp": 1744089253069349400,  // received ~128ms later
  "loop_timestamp": 1625465627457,           // monotonic loop timestamp (~27 min uptime)
  "loop_delay_nanoseconds": 3660,            // internal scheduling delay ~3.66µs
  "event_loop_delay_nanoseconds": 128349376  // external network delay ~128ms
}

These fields facilitate:

Debugging of latency issues.

Monitoring scheduling jitter.

Quantifying end-to-end event processing performance.
"""
async def handle_user_fills(msg, address):
    await rate_limiter.acquire_address_action(address)

    arrival_wall_sec = time.time()  # Wall clock time at arrival (seconds since epoch)
    arrival_loop_monotonic_sec = asyncio.get_event_loop().time()  # Monotonic arrival (loop start based)

    fills = msg.get("data", {}).get("fills", [])
    for f in fills:
        try:
            ts_ms = f.get("time")  # milliseconds since epoch from exchange
            exchange_timestamp_ns = int(ts_ms * 1_000_000)  # ms → ns

            processing_loop_monotonic_ns = int(asyncio.get_event_loop().time() * 1_000_000_000)  # monotonic ns
            arrival_loop_monotonic_ns = int(arrival_loop_monotonic_sec * 1_000_000_000)  # monotonic arrival ns
            receipt_timestamp_ns = int(arrival_wall_sec * 1_000_000_000)  # wall-clock arrival ns

            # Correct delays:
            loop_delay_ns = processing_loop_monotonic_ns - arrival_loop_monotonic_ns
            event_loop_delay_ns = receipt_timestamp_ns - exchange_timestamp_ns  # pure epoch-based difference

            liquidation = f.get("liquidation", {}) or {}
            is_liquidation = 1 if liquidation else 0
            liquidated_user = liquidation.get("liquidatedUser") if is_liquidation else -1
            liquidation_method = liquidation.get("method") if is_liquidation else None
            liquidation_mark_price = liquidation.get("markPx") if is_liquidation else -1

            normalized_side = {"B": "buy", "A": "sell"}.get(f.get("side"), f.get("side"))

            fill_data = {
                "exchange": "hyperliquid",
                "symbol": f.get("coin"),
                "side": normalized_side,
                "price": f.get("px"),
                "amount": f.get("sz"),
                "fee": f.get("fee"),
                "id": f.get("hash"),
                "order_id": f.get("oid"),
                "liquidity": f.get("crossed"),
                "type": f.get("dir"),
                "account": address,

                "timestamp": exchange_timestamp_ns,  # exchange time (epoch ns)
                "receipt_timestamp": receipt_timestamp_ns,  # wall clock (epoch ns)
                "loop_timestamp": processing_loop_monotonic_ns,  # monotonic ns

                "loop_delay_nanoseconds": loop_delay_ns,  # monotonic delay
                "event_loop_delay_nanoseconds": event_loop_delay_ns,  # wall-clock (network) latency

                "is_liquidation": is_liquidation,
                "liquidated_user": liquidated_user,
                "liquidation_method": liquidation_method,
                "liquidation_mark_price": liquidation_mark_price,
                "raw": f,
                "raw_data": f,
            }

            print(f"[DEBUG] Fills data for address {address}: {fill_data}")
            await fill_kafka_cb.write(fill_data)

            if is_liquidation:
                liquidation_data = {
                    "exchange": "hyperliquid",
                    "symbol": f.get("coin"),
                    "side": normalized_side,
                    "quantity": f.get("sz"),
                    "price": f.get("px"),
                    "id": f.get("hash"),
                    "status": f.get("dir"),
                    "timestamp": exchange_timestamp_ns,  # exchange time (epoch ns)
                    "receipt_timestamp": receipt_timestamp_ns,  # wall clock (epoch ns)
                    "loop_timestamp": processing_loop_monotonic_ns,  # monotonic ns

                    "loop_delay_nanoseconds": loop_delay_ns,  # monotonic delay
                    "event_loop_delay_nanoseconds": event_loop_delay_ns,  # wall-clock (network) latency
                    "is_liquidation": is_liquidation,
                    "liquidated_user": liquidated_user,
                    "liquidation_method": liquidation_method,
                    "liquidation_mark_price": liquidation_mark_price,
                    "raw": f,
                    "raw_data": f,
                }
                await liquidation_kafka_cb.write(liquidation_data)

        except Exception as e:
            print(f"[WARN] Failed to process fill for address {address}: {e}")
# async def handle_user_fills(msg, address):
#     await rate_limiter.acquire_address_action(address)
#     fills = msg.get("data", {}).get("fills", [])
#     for f in fills:
#         try:
#             ts = f.get("time")  # Unit: milliseconds since Unix epoch. exchange time.
#             now = time.time()  # Unit: seconds (float) since Unix epoch. wall clock time
#             # loop_timestamp = asyncio.get_event_loop().time()  # Unit: seconds (float) since the event loop started (not related to Unix epoch).
#             loop_timestamp = int(asyncio.get_event_loop().time() * 1_000_000_000) # Unit: nanosecondsseconds (float) since the event loop started (not related to Unix epoch).
 
#             # # Compute delays in nanoseconds
#             # loop_delay_ns = int((now - loop_timestamp) * 1_000_000_000)
#             # event_loop_delay_ns = int((loop_timestamp - (ts / 1000)) * 1_000_000_000)

#             # Extract liquidation info
#             liquidation = f.get("liquidation", {}) or {}
#             is_liquidation = 1 if liquidation else 0
#             liquidated_user = liquidation.get("liquidatedUser") if is_liquidation else -1
#             liquidation_method = liquidation.get("method") if is_liquidation else None
#             liquidation_mark_price = liquidation.get("markPx") if is_liquidation else -1

#             normalized_side = {"B": "buy", "A": "sell"}.get(f.get("side"), f.get("side"))

#             fill_data = {
#                 "exchange": "hyperliquid",
#                 "symbol": f.get("coin"),
#                 "side": normalized_side,
#                 "price": f.get("px"),
#                 "amount": f.get("sz"),
#                 "fee": f.get("fee"),
#                 "id": f.get("hash"),
#                 "order_id": f.get("oid"),
#                 "liquidity": f.get("crossed"),
#                 "type": f.get("dir"),
#                 "account": address,
#                 "timestamp": ts,
#                 "receipt_timestamp": now,
#                 "loop_timestamp": loop_timestamp,
#                 # "loop_delay_nanoseconds": loop_delay_ns,
#                 # "event_loop_delay_nanoseconds": event_loop_delay_ns,
#                 "is_liquidation": is_liquidation,
#                 "liquidated_user": liquidated_user,
#                 "liquidation_method": liquidation_method,
#                 "liquidation_mark_price": liquidation_mark_price,
#                 "raw": f,
#                 "raw_data": f,
#             }

#             print(f"[DEBUG] Fills data for address {address}: {fill_data}")
#             await fill_kafka_cb.write(fill_data)

#             if is_liquidation:
#                 liquidation_data = {
#                     "exchange": "hyperliquid",
#                     "symbol": f.get("coin"),
#                     "side": normalized_side,
#                     "quantity": f.get("sz"),
#                     "price": f.get("px"),
#                     "id": f.get("hash"),
#                     "status": f.get("dir"),
#                     "timestamp": ts,
#                     "receipt_timestamp": now,
#                     "loop_timestamp": loop_timestamp,
#                     # "loop_delay_nanoseconds": loop_delay_ns,
#                     # "event_loop_delay_nanoseconds": event_loop_delay_ns,
#                     "is_liquidation": is_liquidation,
#                     "liquidated_user": liquidated_user,
#                     "liquidation_method": liquidation_method,
#                     "liquidation_mark_price": liquidation_mark_price,
#                     "raw": f,
#                     "raw_data": f,
#                 }
#                 await liquidation_kafka_cb.write(liquidation_data)

#         except Exception as e:
#             print(f"[WARN] Failed to process fill for address {address}: {e}")




async def handle_order_updates(msg, address):
    # Rate limit for each order update.
    await rate_limiter.acquire_address_action(address)
    # now = asyncio.get_event_loop().time()

    arrival_wall_sec = time.time()  # Wall clock time at arrival (seconds since epoch)
    arrival_loop_monotonic_sec = asyncio.get_event_loop().time()  # Monotonic arrival (loop 

    for update in msg.get("data", []):
        order = update.get("order", {})
        side = order.get("side")
        normalized_side = {"B": "buy", "A": "sell"}.get(side, side)
        ts_ms = update.get("time")  # milliseconds since epoch from exchange
        exchange_timestamp_ns = int(ts_ms * 1_000_000)  # ms → ns

        processing_loop_monotonic_ns = int(asyncio.get_event_loop().time() * 1_000_000_000)  # monotonic ns
        arrival_loop_monotonic_ns = int(arrival_loop_monotonic_sec * 1_000_000_000)  # monotonic arrival ns
        receipt_timestamp_ns = int(arrival_wall_sec * 1_000_000_000)  # wall-clock arrival ns

        # Correct delays:
        loop_delay_ns = processing_loop_monotonic_ns - arrival_loop_monotonic_ns
        event_loop_delay_ns = receipt_timestamp_ns - exchange_timestamp_ns  # pure epoch-based difference

        # Compute remaining amount if both origSz and sz are available
        try:
            orig_sz = float(order.get("origSz", "nan"))
            current_sz = float(order.get("sz", "nan"))
            remaining_amount = orig_sz - current_sz
        except Exception:
            remaining_amount = None  # fallback if values are missing or invalid

        order_data = {
            "exchange": "hyperliquid",
            "symbol": order.get("coin"),
            "id": order.get("oid"),
            "client_order_id": order.get("cloid"),
            "side": normalized_side,
            "status": update.get("status"),
            "type": "limit" if order.get("limitPx") else "unknown",
            "price": order.get("limitPx"),
            "amount": order.get("sz"),
            "remaining_amount": str(remaining_amount) if remaining_amount is not None else None,
            "account": address,
            "timestamp": exchange_timestamp_ns,  # exchange time (epoch ns)
            "receipt_timestamp": receipt_timestamp_ns,  # wall clock (epoch ns)
            "loop_timestamp": processing_loop_monotonic_ns,  # monotonic ns
            "loop_delay_nanoseconds": loop_delay_ns,  # monotonic delay
            "event_loop_delay_nanoseconds": event_loop_delay_ns,  # wall-clock (network) latency
            # "timestamp": ts,
            # "receipt_timestamp": now,
            # "loop_timestamp": loop_timestamp,
            # "loop_delay_nanoseconds": loop_delay_ns,
            # "event_loop_delay_nanoseconds": event_loop_delay_ns,
            "raw": order,
            "raw_data": msg
        }

        print(f"[DEBUG] order_data for address {address}: {order_data}")
        await order_kafka_cb.write(order_data)


# async def backfill_fills(address: str, _start_time: int):
#     _, info, _ = example_utils.setup(constants.MAINNET_API_URL)

#     # 3-day window (in milliseconds)
#     end_time = int(time.time() * 1000)
#     start_time = end_time - 7 * 86400 * 1000  # 7 days ago

#     seen_fill_ids: set[str] = set()

#     try:
#         fills = info.user_fills_by_time(address, start_time, end_time)
#         for msg in fills:
#             fill_id = msg.get("hash")
#             if fill_id in seen_fill_ids:
#                 continue
#             seen_fill_ids.add(fill_id)
#             await handle_user_fills({"data": {"fills": [msg]}}, address)
#             await asyncio.sleep(0.01)  # throttle per fill
#     except Exception as e:
#         print(f"[ERROR] backfill_fills failed for {address}: {e}")


#
# NEW FUNCTION:
# Subscribe a *batch* of addresses under a single `info` (hence one websocket).
#
async def subscribe_batch_of_addresses(address_batch: list[str], include_backfill_fills: bool):
    try:
        # Create one websocket connection for these 10 addresses
        _, info, _ = example_utils.setup(constants.MAINNET_API_URL)

        # Subscribe each address on the same connection
        for address in address_batch:
            await rate_limiter.acquire_address_action(address)
            # if include_backfill_fills:
            #     await backfill_fills(address, start_time)

            info.subscribe(
                {"type": "userFills", "user": address},
                lambda msg: run_coroutine_threadsafe(handle_user_fills(msg, address), event_loop)
            )
            # info.subscribe(
            #     {"type": "orderUpdates", "user": address},
            #     lambda msg: run_coroutine_threadsafe(handle_order_updates(msg, address), event_loop)
            # )

            print(f"[{address}] Subscribed on a shared websocket. Listening...")

        # Keep the connection for this batch active forever
        while True:
            await asyncio.sleep(5)

    except Exception as e:
        # If anything fails, retry
        print(f"[ERROR] Batch subscription failed {address_batch}: {e}")
        await asyncio.sleep(3)
        return await subscribe_batch_of_addresses(address_batch, include_backfill_fills)


#
# MAIN method:
# - ensures topics exist
# - loads addresses
# - for every 10 addresses, launches one subscription task (one websocket).
#

# NOTE; NO BACKFILL TAKING UP SPACE IN THE SOCKET, NO ORDERS. PURELY NET NEW FILLS. 
# WORTH DOING 1 backfill

# async def main():
#     INCLUDE_BACKFILL_FILLS = False
#     ensure_topics(bootstrap_servers, username, password)

async def main():
    global fill_kafka_cb, order_kafka_cb, liquidation_kafka_cb
    USE_DEV = False
    INCLUDE_BACKFILL_FILLS = False

    ensure_topics(bootstrap_servers, username, password)

    fill_kafka_cb = ClickHouseFillKafka(
        bootstrap=KAFKA_SERVER,
        username=USERNAME,
        password=PASSWORD
    )

    order_kafka_cb = ClickHouseOrderKafka(
        bootstrap=KAFKA_SERVER,
        username=USERNAME,
        password=PASSWORD
    )

    if USE_DEV:
        liquidation_kafka_cb = ClickHouseLiquidationsDevKafka(
            bootstrap=KAFKA_SERVER,
            username=USERNAME,
            password=PASSWORD
        )
    else:
        liquidation_kafka_cb = ClickHouseLiquidationsKafka(
            bootstrap=KAFKA_SERVER,
            username=USERNAME,
            password=PASSWORD
        )

    addresses = load_addresses_from_config()

    BATCH_SIZE = 10
    tasks = []  # We'll collect one task per batch, so they all run concurrently.

    for i in range(0, len(addresses), BATCH_SIZE):
        batch = addresses[i : i + BATCH_SIZE]
        print(f"[INFO] Creating a subscription task for addresses {batch}")
        t = asyncio.create_task(subscribe_batch_of_addresses(batch, INCLUDE_BACKFILL_FILLS))
        tasks.append(t)

    # Keep the main alive while all batch tasks run forever
    await asyncio.gather(*tasks)

# Uncomment if you want to run directly:
# if __name__ == "__main__":
#     asyncio.run(main())

# We'll simply call main() here:
await main()


